In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import descartes
from shapely.geometry import Polygon
import geopandas as gpd
import os.path
from PIL import Image
import rasterio

## global variables
CLIP = False
INTERSEC = True

#set working directory
os.chdir("/Users/aminaly/Box Sync/mountain_biodiversity")



In [ ]:
## Read in all the files
kba = gpd.read_file(os.getcwd() + "/data/KBA/KBA2020/KBAsGlobal_2020_September_02_POL.shp")
wdpa = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_Jun2021_Public_shp/WDPA_Jun2021_Public/WDPA_Jun2021_Public_flattened.shp")
gmba = gpd.read_file(os.getcwd() + "/data/GMBA/Gmba_Inventory_GME_210420_Sel_292_attr/Gmba_Inventory_GME_210420_Sel_292_attr.shp")
    

In [14]:
 if(CLIP):
    
    #list of ISOs to use to clip kba & wdpa
    wrld_cntries = ['CHE', 'DEU', 'GHA', 'KOR', 'ZAF']
    
    #clip kba and wdpa using the list of isos 
    kba_c = kba[kba['ISO3'].isin(wrld_cntries)]
    wdpa = wdpa[wdpa['ISO3'].isin(wrld_cntries)]
    
    #gmba will be clipped a little differently. Doesn't have ISOs so we'll use a world shapefile
    world = gpd.read_file(os.getcwd() + "/data/World/world_shp/world.shp")
    world = world[world['CNTRY_NAME'].isin(kba_c['Country'].unique())] 
    gmba_c = gpd.overlay(gmba, world, how="intersection")
    #then we find a list of all the ranges included in the clip, and select those specifically from the main gmba
    gmba_c = gmba[gmba.GMBA_V2_ID.isin(gmba_c.GMBA_V2_ID)]
    
    #Once we've clipped them, save them out as shapefiles
    kba_c.to_file(os.getcwd() + "/data/KBA/KBA2020/clipped_KBAsGlobal_2020_September_02_POL.shp", 
                        driver='ESRI Shapefile')

    wdpa.to_file(os.getcwd() + 
                         "/data/WDPA/WDPA_Jun2021_Public_shp/WDPA_Jun2021_Public/clipped_WDPA_Jun2021_Public_flattened.shp",
                       driver='ESRI Shapefile')

    gmba_c.to_file(os.getcwd() + "/data/GMBA/Gmba_Inventory_GME_210420_Sel_292_attr/clipped_Gmba_Inventory_GME_210420_Sel_292_attr.shp", 
                        driver='ESRI Shapefile')

In [2]:
## This section creates an intersection between the KBA and GMBA, splitting up the KBAs and only keeping
## the parts that overlap

if(CLIP):
    gmba_kba = gpd.overlay(kba_c, gmba_c, how = "intersection")
    gmba_kba.to_file(os.getcwd() + "/data/Combined/clipped_gmba_kba.shp", driver = 'ESRI Shapefile')
else: 
    gmba_kba = gpd.overlay(kba, gmba, how = "intersection")
    gmba_kba.to_file(os.getcwd() + "/data/Combined/gmba_kba.shp", driver = 'ESRI Shapefile')


In [6]:
## This section uses the above intersection to pull out all the KBAs that were included in full intersection
## and we create a new KBA layer that is a subset of the original KBA file with the whole KBA polygon
##(no clipping to mountain range)
#read in KBA
kba = gpd.read_file(os.getcwd() + "/data/KBA/KBA2020/KBAsGlobal_2020_September_02_POL.shp")

# read in clipped intersection
gmba_kba_clipped = gpd.read_file(os.getcwd() + "/data/Combined/clipped_gmba_kba.shp")

#filter original kba by the SiteRecIds that show up & save
gmba_clipped_kba_full = kba[kba.SitRecID.isin(gmba_kba_clipped.SitRecID.unique())]
gmba_clipped_kba_full.to_file(os.getcwd() + "/data/Combined/clipped_full_kba_gmba_kba.shp", driver = 'ESRI Shapefile')

## Now do this for the non-clipped file
# read in unclipped intersection
gmba_kba_clipped = gpd.read_file(os.getcwd() + "/data/Combined/gmba_kba.shp")

#filter original kba by the SiteRecIds that show up
gmba_kba_full = kba[kba.SitRecID.isin(gmba_kba_clipped.SitRecID.unique())]

# save out
gmba_kba_full.to_file(os.getcwd() + "/data/Combined/full_kba_gmba_kba.shp", driver = 'ESRI Shapefile')

In [5]:
gmba_kba_clipped.head()

,SitRecID,Country,ISO3,NatName,IntName,SitArea,IbaStatus,KbaStatus,AzeStatus,AddedDate,...,Level_2,Level_3,Level_4,Level_5,Level_6,Level_7,Level_8,Level_9,Level_10,geometry
0,3261,Switzerland,CHE,Untersee: Ermatinger Becken,Lake Constance: Bay of Ermatingen,458,confirmed,None,None,2008-02-19,...,Europe,Central European Highlands,Central Uplands,None,None,None,None,None,None,"MULTIPOLYGON (((9.12917 47.66667, 9.12985 47.6..."
1,3262,Switzerland,CHE,Untersee-Ende und Rhein bis BibermÃ¼hle,End of Lake Constance and river Rhine until Bi...,368,confirmed,None,None,2008-02-19,...,Europe,Central European Highlands,Central Uplands,None,None,None,None,None,None,"MULTIPOLYGON (((8.87917 47.65446, 8.88085 47.6..."
2,3273,Switzerland,CHE,Tafeljura-Landschaft Baselland - Solothurn,Jura mountains of Baselland - Solothurn,37277,confirmed,None,None,2008-02-19,...,Europe,Central European Highlands,Central Uplands,None,None,None,None,None,None,"MULTIPOLYGON (((7.91042 47.57142, 7.91054 47.5..."
3,3342,Germany,DEU,Bastauniederung,Bastau lowlands,3583,confirmed,None,None,2008-06-13,...,Europe,Central European Highlands,Central Uplands,None,None,None,None,None,None,"POLYGON ((8.67174 52.34627, 8.67187 52.34227, ..."
4,3343,Germany,DEU,Rheinaue Bingen-Ingelheim,Rheinaue Bingen-Ingelheim,3972,confirmed,None,None,2008-06-13,...,Europe,Central European Highlands,Central Uplands,None,None,None,None,None,None,"POLYGON ((8.18987 50.04334, 8.19022 50.04149, ..."
